# ADF test 

In [ ]:
import yfinance as yf
import numpy as np
import pandas as pd
from statsmodels.tsa.stattools import adfuller
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.stats.diagnostic import acorr_ljungbox, het_arch
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from arch import arch_model

# Download S&P 500 historical data (ticker: ^GSPC)
data = yf.download('^GSPC', start='2000-01-01', end='2020-12-31')

# Drop rows with missing values
data = data.dropna()

# Calculate log returns from 'Adj Close' prices
data['LogReturn'] = np.log(data['Close'] / data['Close'].shift(1))

# Drop NaNs resulting from the shift
log_returns = data['LogReturn'].dropna()

import matplotlib.pyplot as plt

plt.plot(log_returns)
plt.title('Log Returns of S&P 500 (2000-2020)')  # Add title
plt.xlabel('Date')  # Add x-axis label
plt.ylabel('Log Return')  # Add y-axis label
plt.savefig('log_returns_plot.png')

adf_result = adfuller(log_returns)

print('ADF Statistic: %f' % adf_result[0])
print('p-value: %f' % adf_result[1])
print('Critical Values:')
for key, value in adf_result[4].items():
    print('\t%s: %.3f' % (key, value))

if adf_result[1] <= 0.05:
    print("Reject the null hypothesis: Time series is stationary")
else:
    print("Fail to reject the null hypothesis: Time series is non-stationary")

In [ ]:
# --- 1. ARMA Model Fitting ---
# Choose ARMA(1,0,1) as example; adjust (p,d,q) as needed
arma_order = (1, 0, 1)
arma_model = ARIMA(log_returns, order=arma_order).fit()
print("\nARMA Model Summary:")
print(arma_model.summary())

# Extract ARMA parameters for table
arma_params = arma_model.params
arma_bse = arma_model.bse
arma_tvalues = arma_model.tvalues
arma_pvalues = arma_model.pvalues

arma_table = pd.DataFrame({
    "Parameter": arma_params.index,
    "Estimate": arma_params.values,
    "Std. Error": arma_bse.values,
    "t-value": arma_tvalues.values,
    "p-value": arma_pvalues.values
})

# --- 2. Residual Diagnostics ---
# Ljung-Box test for residual autocorrelation
ljung_box = acorr_ljungbox(arma_model.resid, lags=[10], return_df=True)
print("\nLjung-Box test for residual autocorrelation:")
print(ljung_box)

# ARCH-LM test for ARCH effects
arch_test_stat, arch_test_pvalue, _, _ = het_arch(arma_model.resid)
print(f"\nARCH-LM Test Statistic: {arch_test_stat:.4f}, p-value: {arch_test_pvalue:.4f}")

arch_table = pd.DataFrame({
    "Test Statistic": [arch_test_stat],
    "p-value": [arch_test_pvalue]
})

# --- 3. GARCH(1,1) Model Fitting ---
# Fit GARCH(1,1) with zero mean (assuming ARMA captured mean)
garch_model = arch_model(log_returns, vol='Garch', p=1, q=1, mean='Zero', dist='normal')
garch_fit = garch_model.fit(update_freq=5, disp='off')
print("\nGARCH(1,1) Model Summary:")
print(garch_fit.summary())

# Extract GARCH parameters for table
garch_params = garch_fit.params
garch_bse = garch_fit.std_err
garch_tvalues = garch_fit.tvalues
garch_pvalues = garch_fit.pvalues

garch_table = pd.DataFrame({
    "Parameter": garch_params.index,
    "Estimate": garch_params.values,
    "Std. Error": garch_bse.values,
    "t-value": garch_tvalues.values,
    "p-value": garch_pvalues.values
})

# --- 4. Plotting ---

# a) Time Series Plot of Data
plt.figure(figsize=(12, 4))
plt.plot(log_returns)
plt.title('Time Series Plot of Stationary Data')
plt.xlabel('Time')
plt.ylabel('Value')
plt.tight_layout()
plt.show()

# b) ACF and PACF plots
fig, ax = plt.subplots(1, 2, figsize=(14, 4))
plot_acf(log_returns, lags=40, ax=ax[0])
ax[0].set_title('ACF of Data')
plot_pacf(log_returns, lags=40, ax=ax[1])
ax[1].set_title('PACF of Data')
plt.tight_layout()
plt.savefig('acf_pacf_plot.png')

# c) Residual and Squared Residual Plots from ARMA
fig, ax = plt.subplots(2, 1, figsize=(12, 6))
ax[0].plot(arma_model.resid)
ax[0].set_title('ARMA Model Residuals')
ax[0].set_xlabel('Time')
ax[0].set_ylabel('Residuals')

ax[1].plot(arma_model.resid**2)
ax[1].set_title('Squared Residuals from ARMA Model')
ax[1].set_xlabel('Time')
ax[1].set_ylabel('Squared Residuals')

plt.tight_layout()
plt.savefig('arma_resid_sqresid.png')

# d) Conditional Volatility Plot from GARCH Model
plt.figure(figsize=(12, 4))
plt.plot(garch_fit.conditional_volatility)
plt.title('Conditional Volatility from GARCH(1,1) Model')
plt.xlabel('Time')
plt.ylabel('Volatility')
plt.tight_layout()
plt.savefig('garch_cond_vol.png')

# e) Standardized Residuals and Squared Standardized Residuals from GARCH
std_resid = garch_fit.std_resid

fig, ax = plt.subplots(2, 1, figsize=(12, 6))
ax[0].plot(std_resid)
ax[0].set_title('Standardized Residuals from GARCH Model')
ax[0].set_xlabel('Time')
ax[0].set_ylabel('Std Residuals')

ax[1].plot(std_resid**2)
ax[1].set_title('Squared Standardized Residuals from GARCH Model')
ax[1].set_xlabel('Time')
ax[1].set_ylabel('Squared Std Residuals')

plt.tight_layout()
plt.savefig('garch_std_resid.png')

# 